<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M2-hatespeech-nlp-explainer-tm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trigger warning: This notebook contains words or language that are considered profane, vulgar, or offensive by some.

In [1]:
!pip install tweet-preprocessor -q

# Installing Gensim and PyLDAvis
!pip install -qq -U gensim
!pip install -qq pyLDAvis

     |████████████████████████████████| 24.1 MB 1.5 MB/s 
     |████████████████████████████████| 1.7 MB 9.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done


In [2]:
# explainability (why did the model say it's hate speech)
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 16.5 MB/s 
     |████████████████████████████████| 133 kB 56.7 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=f70e423022a833dedcfa66b4b8a56c5511df9abe8927d8cdbf54c8c6043dc84b
  Stored in directory: /root/.cache/pip/wheels/cc/3c/96/3ead31a8e6c20fc0f1a707fde2e05d49a80b1b4b30096573be
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.


In [3]:
import pandas as pd
import numpy as np
import preprocessor as prepro # twitter prepro
import tqdm #progress bar

import spacy #spacy for quick language prepro
nlp = spacy.load('en_core_web_sm') #instantiating English module

# sampling, splitting
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split


# loading ML libraries
from sklearn.pipeline import make_pipeline #pipeline creation
from sklearn.feature_extraction.text import TfidfVectorizer #transforms text to sparse matrix
from sklearn.linear_model import LogisticRegression #Logit model
from sklearn.metrics import classification_report #that's self explanatory
from sklearn.decomposition import TruncatedSVD #dimensionality reduction
from xgboost import XGBClassifier

import altair as alt #viz

#explainability
import eli5
from eli5.lime import TextExplainer

# topic modeling

from gensim.corpora.dictionary import Dictionary # Import the dictionary builder
from gensim.models import LdaMulticore # we'll use the faster multicore version of LDA

# Import pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

%matplotlib inline
pyLDAvis.enable_notebook()

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [4]:
# prepro settings
prepro.set_options(prepro.OPT.URL, prepro.OPT.NUMBER, prepro.OPT.RESERVED, prepro.OPT.MENTION, prepro.OPT.SMILEY)

In [5]:
data = pd.read_csv('https://github.com/SDS-AAU/SDS-master/raw/master/M2/data/twitter_hate.zip')

In [7]:
data['text_clean'] = data['tweet'].map(lambda t: prepro.clean(t))
data['text_clean'] = data['text_clean'].str.replace('#','')

Spacy basics

In [24]:
text = "this is a sentence about how much i would like to have a break in spanish they call it pausa"

In [11]:
text_list = text.split(' ')

In [15]:
' '.join(text_list)

'this is a sentence about how much i would like to have a break'

In [25]:
text_spacy = nlp(text)

In [26]:
[(word.lemma_, word.pos_) for word in text_spacy if not word.is_stop]

[('sentence', 'NOUN'),
 ('like', 'VERB'),
 ('break', 'NOUN'),
 ('spanish', 'ADJ'),
 ('pausa', 'NOUN')]

In [27]:
text_spacy.vector

array([ 0.3401806 ,  0.19296041,  0.14848831,  0.05172095,  0.16029048,
       -0.33672786, -0.1995562 ,  0.10831702,  0.28926075,  0.1138801 ,
        0.05822397, -0.23683909, -0.17793842, -0.00266089, -0.06673323,
        0.27064067, -0.3068571 , -0.09510701, -0.18617156, -0.19235735,
        0.24233618,  0.09865516,  0.01819545, -0.16571267,  0.14841723,
       -0.11937156,  0.2731852 , -0.15597975, -0.19402547,  0.0787922 ,
       -0.3629102 , -0.19420913, -0.43860024, -0.12403061, -0.41017637,
        0.2885998 , -0.4366033 , -0.21358559,  0.20607474,  0.26243302,
        0.29514796,  0.5686544 ,  0.44046718, -0.41974887, -0.07641989,
        0.6284071 ,  0.32461512, -0.03137162, -0.21161003,  0.04054604,
        0.44923908, -0.3466888 , -0.04890352,  0.4222819 , -0.10918555,
       -0.04555248,  0.24646029,  0.0660795 , -0.2912985 ,  0.10088477,
       -0.22141394, -0.28841218,  0.1168973 , -0.12291601, -0.28370205,
       -0.25856125, -0.38925764, -0.0804797 , -0.08510756, -0.34

In [37]:
text_spacy = nlp('Aalborg Univ. is a nice place where Donaldo Trump has never been.')

In [38]:
[(token, token.label_) for token in text_spacy.ents]

[(Aalborg Univ, 'ORG'), (Donaldo Trump, 'ORG')]

back to the project

In [39]:
# run progress bar and clean up using spacy but without some heavy parts of the pipeline

clean_text = []

pbar = tqdm.tqdm(total=len(data['text_clean']),position=0, leave=True)

for text in nlp.pipe(data['text_clean'], disable=["tagger", "parser", "ner"]):

  txt = [token.lemma_.lower() for token in text 
         if token.is_alpha 
         and not token.is_stop 
         and not token.is_punct]

  clean_text.append(" ".join(txt))

  pbar.update(1)

  0%|          | 0/24783 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
 99%|█████████▉| 24577/24783 [00:26<00:00, 1150.19it/s]

In [41]:
# write everything into one function that can be re-used later
def text_prepro(texts):
  """
  takes in a pandas series (1 column of a DF)
  removes twitter stuff
  lowercases, normalizes text
  """
  texts_clean = texts.map(lambda t: prepro.clean(t))
  texts_clean = texts_clean.str.replace('#','')

  clean_container = []

  pbar = tqdm.tqdm(total=len(texts_clean),position=0, leave=True)

  for text in nlp.pipe(texts_clean, disable=["tagger", "parser", "ner"]):

    txt = [token.lemma_.lower() for token in text 
          if token.is_alpha 
          and not token.is_stop 
          and not token.is_punct]

    clean_container.append(" ".join(txt))
    pbar.update(1)
  
  return clean_container

In [42]:
# apply all prepro-pipeline to texts
data['text_clean'] = text_prepro(data['tweet'])

100%|██████████| 24783/24783 [00:23<00:00, 1059.74it/s]


In [43]:
data

,Unnamed: 0,class,tweet,text_clean
0,0,2,!!! RT @mayasolovely: As a woman you shouldn't...,rt woman complain cleaning house amp man trash
1,1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,rt boy dats cold tyga dwn bad cuffin dat hoe s...
2,2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,rt dawg rt fuck bitch start cry confused shit
3,3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,rt look like tranny
4,4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,rt shit hear true faker bitch told ya
...,...,...,...,...
24778,24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,lie right tl trash bible scriptures
24779,24779,2,"you've gone and broke the wrong heart baby, an...",gone broke wrong heart baby drove redneck crazy
24780,24780,1,young buck wanna eat!!.. dat nigguh like I ain...,young buck wanna eat dat nigguh like ai nt fuc...
24781,24781,1,youu got wild bitches tellin you lies,youu got wild bitches tellin lies


In [45]:
# renaming and reordering

data_df = pd.DataFrame({'label':data['class'], 'text':data['text_clean']})

In [46]:

data_df.label.value_counts().reset_index()

,index,label
0,1,19190
1,2,4163
2,0,1430


In [47]:
alt.Chart(data_df.label.value_counts().reset_index()).mark_bar(filled=True).encode(
    alt.X('label:Q', title='N Tweets'),
    alt.Y('index:N', title='Category')
)

alt.Chart(...)

In [48]:
# fixing sample imbalance
rus = RandomUnderSampler(random_state=42)
data_df_res, y_res = rus.fit_resample(data_df, data_df['label'])

In [49]:
data_df_res['label'].value_counts()

0    1430
1    1430
2    1430
Name: label, dtype: int64

In [50]:
# Splitting the dataset into the Training set and Test set (since we have a new output variable)
X_train, X_test, y_train, y_test = train_test_split(data_df_res['text'], y_res, test_size = 0.4, random_state = 42)

In [51]:
#instantiate models and "bundle up as pipeline"

tfidf = TfidfVectorizer()
cls = LogisticRegression()

pipe = make_pipeline(tfidf, cls)

In [52]:
pipe.fit(X_train,y_train) # fit model

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression', LogisticRegression())])

In [53]:
# evaluate model performance on training set

y_eval = pipe.predict(X_train)
report = classification_report(y_train, y_eval)
print(report)

              precision    recall  f1-score   support

           0       0.94      0.87      0.90       840
           1       0.91      0.91      0.91       842
           2       0.94      0.99      0.97       892

    accuracy                           0.93      2574
   macro avg       0.93      0.93      0.93      2574
weighted avg       0.93      0.93      0.93      2574



In [55]:
# run single prediction

t1 = ['you stupid fag bitch']

In [56]:
# preprocess

t1_p = text_prepro(pd.Series(t1)) # note, we need to pack text up as pd.Series 

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
100%|██████████| 1/1 [00:00<00:00, 85.77it/s]


In [58]:
# predict

pipe.predict(t1_p)

array([0])

In [59]:
# overall weights (works only for linear models)
eli5.show_weights(pipe, top=10, target_names=['hate','offensive','nothing'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [60]:
# explain one prediction
eli5.show_prediction(pipe[1], t1_p[0], vec=pipe[0],
                     target_names=['hate','offensive','nothing'])

In [61]:
data['tweet'][100]

'"@ClicquotSuave: LMAOOOOOOOOOOO this nigga @Krillz_Nuh_Care http://t.co/AAnpSUjmYI" &lt;bitch want likes for some depressing shit..foh'

In [62]:
data['class'][100]

1

In [63]:
eli5.show_prediction(pipe[1], data['text_clean'][100], vec=pipe[0],
                     target_names=['hate','offensive','nothing'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Let's try a complex (black-box) model

In [66]:
#instantiate models and "bundle up as pipeline"

tfidf = TfidfVectorizer()
svd = TruncatedSVD(n_components = 100)
cls_xg = XGBClassifier()

pipe_xg = make_pipeline(tfidf, svd, cls_xg)

In [67]:
pipe_xg.fit(X_train,y_train) # fit model

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('truncatedsvd', TruncatedSVD(n_components=100)),
                ('xgbclassifier', XGBClassifier(objective='multi:softprob'))])

In [68]:
# evaluate model performance on training set

y_eval = pipe_xg.predict(X_train)
report = classification_report(y_train, y_eval)
print(report)

              precision    recall  f1-score   support

           0       0.91      0.78      0.84       840
           1       0.89      0.87      0.88       842
           2       0.85      0.98      0.91       892

    accuracy                           0.88      2574
   macro avg       0.88      0.88      0.88      2574
weighted avg       0.88      0.88      0.88      2574



In [69]:
# evaluate model performance on test set

y_pred = pipe_xg.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.76      0.63      0.69       590
           1       0.76      0.77      0.77       588
           2       0.76      0.89      0.82       538

    accuracy                           0.76      1716
   macro avg       0.76      0.76      0.76      1716
weighted avg       0.76      0.76      0.76      1716



In [70]:
# explain single prediction
te = TextExplainer(random_state=42)
te.fit(data['text_clean'][100], pipe_xg.predict_proba)
te.show_prediction(target_names=['hate','offensive','nothing'])

/usr/local/lib/python3.7/dist-packages/eli5/lime/samplers.py:191: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  for idx, freq in itemfreq(sampler_indices)
/usr/local/lib/python3.7/dist-packages/eli5/lime/textutils.py:186: DeprecationWarning: Flags not at the start of the expression '((?u)\\b\\w+\\b)'
  parts = re.split(token_pattern, text)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [88]:
pred_xg = pipe_xg.predict(X_test)


In [91]:
X_test_evaluation =pd.DataFrame(X_test)

In [93]:
X_test_evaluation.index = range(len(X_test_evaluation))

In [96]:
X_test_evaluation['y_test'] = list(y_test)

In [97]:
X_test_evaluation['pred_xg'] = list(pred_xg)

In [98]:
X_test_evaluation

,text,y_test,pred_xg
0,happy birthday bob newhart comedic legend turn...,2,2
1,msnbc msm said blocking flights africa racist ...,1,1
2,word bird,2,2
3,good things come end right tomorrow yankees,2,2
4,daniel cow tipping hoe downs jackson city livi...,2,1
...,...,...,...
1711,leave dyke,0,0
1712,bitches lobby plenty bitches lobby,1,1
1713,somebody choke fucking retard keeps yelling ma...,0,0
1714,convince u lame nigger liver believes cuz nigg...,0,0


In [100]:
pd.crosstab(X_test_evaluation['y_test'],X_test_evaluation['pred_xg'], normalize='index')

pred_xg,0,1,2
y_test,,,
0,0.633898,0.181356,0.184746
1,0.156463,0.767007,0.076531
2,0.053903,0.059480,0.886617


## Topic model

In [71]:
# preprocess texts (we need tokens)
tokens = []

for text in nlp.pipe(data['text_clean'], disable=["ner"]):
  proj_tok = [token.lemma_.lower() for token in text 
              if token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV'] 
              and not token.is_stop
              and not token.is_punct] 
  tokens.append(proj_tok)

In [75]:
data['tokens'] = tokens

In [76]:
data

,Unnamed: 0,class,tweet,text_clean,tokens
0,0,2,!!! RT @mayasolovely: As a woman you shouldn't...,rt woman complain cleaning house amp man trash,"[rt, woman, house, amp, man, trash]"
1,1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,rt boy dats cold tyga dwn bad cuffin dat hoe s...,"[rt, boy, cold, tyga, dwn, bad, cuffin, hoe, s..."
2,2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,rt dawg rt fuck bitch start cry confused shit,"[rt, dawg, rt, fuck, bitch, confused, shit]"
3,3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,rt look like tranny,[tranny]
4,4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,rt shit hear true faker bitch told ya,"[rt, shit, true, faker, bitch, ya]"
...,...,...,...,...,...
24778,24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,lie right tl trash bible scriptures,"[right, tl, trash, bible, scripture]"
24779,24779,2,"you've gone and broke the wrong heart baby, an...",gone broke wrong heart baby drove redneck crazy,"[wrong, heart, baby, redneck, crazy]"
24780,24780,1,young buck wanna eat!!.. dat nigguh like I ain...,young buck wanna eat dat nigguh like ai nt fuc...,"[young, buck, wanna, dat, nigguh, fuckin, di]"
24781,24781,1,youu got wild bitches tellin you lies,youu got wild bitches tellin lies,"[youu, wild, bitch, tellin]"


We would like to know what things people are talking about when it is considerede "hatespeech"

In [77]:
data_hate = data[data['class'] == 0]

In [78]:
# Create a Dictionary from the articles: dictionary
dictionary = Dictionary(data_hate['tokens'])
# filter out low-frequency / high-frequency stuff, also limit the vocabulary to max 1000 words
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)
# construct corpus using this dictionary
corpus = [dictionary.doc2bow(doc) for doc in data_hate['tokens']]

In [82]:
# Training the model
lda_model = LdaMulticore(corpus, id2word=dictionary, num_topics=10, workers = 4, passes=10)

In [83]:
# Let's try to visualize
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [86]:
 # Let's Visualize
pyLDAvis.display(lda_display)